In [118]:
import os
import numpy as np
import pandas as pd

Connect to Google Drive

In [119]:
#from google.colab import drive
#drive.mount('/content/drive')
#disk_path = os.path.join('drive/My Drive/CNN_Data')

Input the version names for comparison

In [120]:
#disk_path = 'F:/server_output/processed/'
disk_path = 'E:/Diane/Col/research/code/Done/Processed/version_output'
all_version_name = np.sort(os.listdir(disk_path))
#for version_name in all_version_name:
version1='v11.007'
version2='v14.21'

### Define some functions



In [121]:
def check_foldername(foldernames,test_string):
    has_test_string = np.zeros((len(foldernames),len(test_string)))
    for f_i,foldername in enumerate(foldernames):
        for s_i,string in enumerate(test_string):
            if string in foldername.lower():
                has_test_string[f_i,s_i] = 1
    #remove foldernames that doesn't contain any element of test_string
    keep_vec = np.sum(has_test_string,1)
    if np.where(keep_vec>0)[0].size < len(foldernames):
        print(f'ignore {len(foldernames)-np.where(keep_vec>0)[0].size} folder')
    foldernames_new = [foldernames[i] for i in np.where(keep_vec>0)[0]]#only keep the folder names with "stem" or "leaf"
    has_test_string_new = np.asarray([has_test_string[i,] for i in np.where(keep_vec>0)[0]])
    return foldernames_new,has_test_string_new

def get_metric_from_line(string):
    '''
    input: "['sensitivity', 62.5), 'precision', 56.18), 'accuracy', 92.32)]\n"
    output: an array of [62.5,56.18,92.32]
    '''
    string2 = string.replace("(","").replace(")","").replace("[","").replace("]","").replace("\n","").replace("'","")
    string_list = string2.split(', ')
    value_vec = np.zeros(int(len(string_list)/2))
    for i in range(0,len(value_vec)):
        value_vec[i] = string_list[1+i*2]
    return value_vec

def clean_folder_name(all_folders_name,has_stem_leaf,stem_leaf_list):
    '''
    input: ['Alclat2_stem.DONEGOOD.HANNAH.9.26', 'casarb2.2_stem.HANNAHDONEGOD.11.4', 'casarb5_stem.DONEGOOD.HANNAH.11.7'...]
    output: ['Alclat2_stem', 'casarb2.2_stem', 'casarb5_stem',...]
    '''
    all_folders_name_short=all_folders_name.copy()
    for i, foldername in enumerate(all_folders_name):
        terminate_word = stem_leaf_list[int(np.where(has_stem_leaf[i]>0)[0])]#stem or leaf
        stop_index = foldername.lower().find(terminate_word)
        all_folders_name_short[i] = foldername[:(stop_index+len(terminate_word))]
    return all_folders_name_short

def color_val(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, green for positive, black otherwise.
    """
    
    if val < -0.1:
        color = 'red'
    elif val > 0.1 :
        color = 'green'
    else:
         color = 'gray'
    return 'color: %s' % color
        

The main function for one version's summary result

In [122]:
def one_version_pd(disk_path,version_name):
    print(f'version {version_name} ...')
    version_dir = os.path.join(disk_path,version_name)
    all_folders_name = sorted(os.listdir(version_dir))
    #print(f'{all_folders_name}')
    #only keep the folder names with "stem" or "leaf" (i.e. ignore folders like "output")
    stem_leaf_list = ["stem","leaf"]
    all_folders_name,has_stem_leaf = check_foldername(all_folders_name,stem_leaf_list)
    #print(f'{has_stem_leaf}')#len(all_folders_name) x 2: each entry is {0 or 1}. 1st col "stem", 2nd col "leaf"
    all_folders_name_short = clean_folder_name(all_folders_name,has_stem_leaf,stem_leaf_list)
    #print(f'{all_folders_name_short}')

    all_folders_dir = [os.path.join(version_dir,folder) for folder in all_folders_name]
    all_folders_dir_processed = [os.path.join(folder_dir, os.listdir(folder_dir)[0]) for folder_dir in all_folders_dir]
    #print(f'{all_folders_dir_processed}')
    all_line_header = ['img level metric','pixel level metric','cluster level metric']
    all_line_header_short = [b.replace(" metric","") for b in all_line_header]
    metric_name_list = ['sensitivity','precision','accuracy']
    metric_num = len(metric_name_list)
    folder_metric_arr = np.zeros((len(all_folders_name),len(all_line_header*metric_num)))

    cm_colname = ['Predict 0', 'Predict 1']
    cm_rowname = ['True 0', 'True 1']
    cm_row = len(cm_rowname)# number of rows in a confusion matrix
    cm_col = len(cm_colname)
    #img_cm = ["" for x in range(len(all_folders_name)*cm_row)]#list
    img_cm = np.zeros((len(all_folders_name)*cm_row,cm_col))
    for p_i, processed_path in enumerate(all_folders_dir_processed):
        folder_name = all_folders_name[p_i]
        #print(f'processing {folder_name} ...')
        print(f'{all_folders_name_short[p_i]}')
        con_mat_path = os.path.join(processed_path,'confusion_mat_file.txt')
        if os.path.exists(con_mat_path):#check if confusion_mat_file.txt exists
            with open (con_mat_path) as file:
                datafile=file.readlines()
                for l_i,line in enumerate(datafile):
                    for lh_i,line_header in enumerate(all_line_header):
                        if line_header in line:
                            st = lh_i*metric_num
                            end = st+metric_num
                            folder_metric_arr[p_i,st:end] = get_metric_from_line(datafile[l_i+1])#"l_i+1":next line of 'img level metric'
                            if line_header == 'img level metric':
                                st = p_i*cm_row
                                for cm_i in range(cm_row):
                                    line_string_list = datafile[l_i+4+cm_i].replace("\n","").split()
                                    p0 = float(line_string_list[2])#predict 0
                                    p1 = float(line_string_list[-1])
                                    img_cm[st+cm_i,:] = [p0,p1]
        else:#if confusion_mat_file.txt doesn't exist, filled up with NaN
            folder_metric_arr[p_i,:] = np.nan
            st = p_i*cm_row
            for cm_i in range(cm_row):
                img_cm[st+cm_i,:] = [np.nan,np.nan]
    #add headers
    column_names_arr = [np.repeat(all_line_header,3).tolist(),(metric_name_list*3)]
    col_tuples = list(zip(*column_names_arr))
    column_names = pd.MultiIndex.from_tuples(col_tuples, names=['level', 'metric'])
    metric_df = pd.DataFrame(folder_metric_arr, columns=column_names, index=all_folders_name_short)

    #img_cm = img_cm.astype(int)#convert from float to int
    #add headers
    column_names_arr_img = [([version_name+' img level con. matrix']*cm_col),cm_colname]
    col_tuples_img = list(zip(*column_names_arr_img))
    column_names_img = pd.MultiIndex.from_tuples(col_tuples_img, names=['version/level', 'prediction/label'])
    row_names_arr_img = [np.repeat(all_folders_name_short,cm_row).tolist(),cm_rowname*len(all_folders_name_short)]
    row_tuples_img = list(zip(*row_names_arr_img))
    row_names_img = pd.MultiIndex.from_tuples(row_tuples_img)
    img_cm_df = pd.DataFrame(img_cm, columns=column_names_img, index=row_names_img)
    return metric_df,img_cm_df

## Compare two versions metric results

In [123]:
metric_df1,img_cm1 = one_version_pd(disk_path,version1)
metric_df2,img_cm2 = one_version_pd(disk_path,version2)

version v11.007 ...
Alclat2_stem
casarb2.2_stem
casarb5_stem
drygla5_stem
inglau3_stem
inglau4_stem
inglau5_stem
version v14.21 ...
Alclat2_stem
casarb2.2_stem
casarb5_stem
drygla5_stem
inglau3_stem
inglau4_stem
inglau5_stem


#### Print out result of each version


In [124]:
print(version1)
metric_df1

v11.007


level          img level metric                    pixel level metric  \
metric              sensitivity precision accuracy        sensitivity   
Alclat2_stem             100.00     13.16    83.42              85.40   
casarb2.2_stem            96.15     47.17    85.43              57.44   
casarb5_stem              85.71     24.49    80.40              58.42   
drygla5_stem              50.00     10.53    90.45               0.00   
inglau3_stem              66.67     44.44    96.48              76.43   
inglau4_stem             100.00     28.85    81.41              65.90   
inglau5_stem              71.43     38.46    94.97              80.21   

level                             cluster level metric                     
metric         precision accuracy          sensitivity precision accuracy  
Alclat2_stem       11.40    99.93               100.00      9.38    74.22  
casarb2.2_stem     43.00    99.83                82.50     42.86    77.73  
casarb5_stem       11.70    99.89                79.31     26.44    70.95  
drygla5_stem        0.00     0.00                 0.00      0.00     0.00  
inglau3_stem       48.62    99.98                45.45     45.45    94.15  
inglau4_stem       24.41    99.93                75.00     21.88    67.20  
inglau5_stem       42.28    99.99                55.56     29.41    92.20

In [125]:
print(version2)
metric_df2

v14.21


level          img level metric                    pixel level metric  \
metric              sensitivity precision accuracy        sensitivity   
Alclat2_stem             100.00     15.62    86.43              84.24   
casarb2.2_stem            96.15     52.08    87.94              57.12   
casarb5_stem              85.71     30.77    85.43              55.32   
drygla5_stem              50.00     20.00    94.97               0.00   
inglau3_stem              66.67     44.44    96.48              71.03   
inglau4_stem             100.00     33.33    84.92              65.55   
inglau5_stem              71.43     50.00    96.48              80.06   

level                             cluster level metric                     
metric         precision accuracy          sensitivity precision accuracy  
Alclat2_stem       10.44    99.92               100.00     11.11    78.28  
casarb2.2_stem     45.70    99.84                80.00     47.76    80.89  
casarb5_stem       16.76    99.93                79.31     38.33    80.80  
drygla5_stem        0.00     0.00                 0.00      0.00     0.00  
inglau3_stem       53.49    99.98                45.45     50.00    94.61  
inglau4_stem       31.63    99.95                75.00     22.83    69.17  
inglau5_stem       47.46    99.99                55.56     35.71    93.66

#### Print out 2 version's difference


In [126]:
diff_md = metric_df2-metric_df1

diff_md_colored = diff_md.style.applymap(color_val)
diff_md_colored

### Print out img level confusion matrix

In [127]:
img_cm_join = pd.concat([img_cm1, img_cm2],axis=1)
img_cm_join

version/level         v11.007 img level con. matrix            \
prediction/label                          Predict 0 Predict 1   
Alclat2_stem   True 0                         161.0      33.0   
               True 1                           0.0       5.0   
casarb2.2_stem True 0                         145.0      28.0   
               True 1                           1.0      25.0   
casarb5_stem   True 0                         148.0      37.0   
               True 1                           2.0      12.0   
drygla5_stem   True 0                         178.0      17.0   
               True 1                           2.0       2.0   
inglau3_stem   True 0                         188.0       5.0   
               True 1                           2.0       4.0   
inglau4_stem   True 0                         147.0      37.0   
               True 1                           0.0      15.0   
inglau5_stem   True 0                         184.0       8.0   
               True 1                           2.0       5.0   

version/level         v14.21 img level con. matrix            
prediction/label                         Predict 0 Predict 1  
Alclat2_stem   True 0                        167.0      27.0  
               True 1                          0.0       5.0  
casarb2.2_stem True 0                        150.0      23.0  
               True 1                          1.0      25.0  
casarb5_stem   True 0                        158.0      27.0  
               True 1                          2.0      12.0  
drygla5_stem   True 0                        187.0       8.0  
               True 1                          2.0       2.0  
inglau3_stem   True 0                        188.0       5.0  
               True 1                          2.0       4.0  
inglau4_stem   True 0                        154.0      30.0  
               True 1                          0.0      15.0  
inglau5_stem   True 0                        187.0       5.0  
               True 1                          2.0       5.0

#### Print out 2 version's difference in img-level confusion matrix

In [128]:
#drop 1st header s.t. they can subtract
new_col_header = img_cm1.columns.droplevel()#only 1 level
img_cm_dr1 = img_cm1.copy()
img_cm_dr1.columns = new_col_header
img_cm_dr2 = img_cm2.copy()
img_cm_dr2.columns = new_col_header
diff_img_cm = img_cm_dr2-img_cm_dr1
#2 cols would be the same w/ sign flipped

diff_img_cm_tn = diff_img_cm.loc[(slice(None), ['True 0']), ['Predict 0']]#change in true negative

diff_img_cm_tp = diff_img_cm.loc[(slice(None), ['True 1']), ['Predict 1']]#change in true positive

#### Change in true positives(img-level)
More true positive means less false negative

In [129]:
diff_img_cm_tp.style.applymap(color_val)

,prediction/label,Predict 1
Alclat2_stem,True 1,0
casarb2.2_stem,True 1,0
casarb5_stem,True 1,0
drygla5_stem,True 1,0
inglau3_stem,True 1,0
inglau4_stem,True 1,0
inglau5_stem,True 1,0


#### Change in true negatives(img-level)
More true negative means less false positive

In [130]:
diff_img_cm_tn.style.applymap(color_val)

,prediction/label,Predict 0
Alclat2_stem,True 0,6
casarb2.2_stem,True 0,5
casarb5_stem,True 0,10
drygla5_stem,True 0,9
inglau3_stem,True 0,0
inglau4_stem,True 0,7
inglau5_stem,True 0,3


14.1 vs 11.007: samd FN, decrease FP (if there's big emb cc but its direction is not the same as stem direction --> no emb)

14.007/14.0 compared with 11.007:
- added remove_cc_by_dir: for strong emb candidate, if there's big cc, keep it if the direction of cc is similar to the stem direction it lies in.


14.1 added new feature (compared with v14.0/14.007): 
remove_cc_by_dir:
1. get_cos_one_cc_and_stem: stem_min_row=stem_est_width*2
2. if no big cc: keep entire img

14.2:
1. input for remove_cc_by_dir (dir_input_stack) changes from input_stack to final_stack_strong_emb_cand, because final_stack_strong_emb_cand has already cleaned some FP (ex: a2's 0,1,2.jpg)

2. dir_cos_min = 0.9 -> 0.95

14.21:
- dir_cos_min: 0.95 -> 0.97
